In [1]:
import os
import datetime, warnings, scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit

from pandas.plotting import scatter_matrix

pd.options.display.max_columns = None

#os.chdir('/Users/Marta/Dropbox/2019-Move-to-NL/UvA/info_viz_course/project')
print(os.getcwd())

/Users/antonkozackov/Yandex.Disk/Yandex.Disk.localized/UvA Data Coursework/Information Visualization/infovis/jupyter


In [3]:
airline_names = pd.read_csv("../app/data/airlines.csv")
airline_names

,IATA_CODE,AIRLINE
0,UA,United Airlines
1,AA,American Airlines
2,US,US Airways
3,F9,Frontier Airlines
4,B6,JetBlue Airways
5,OO,SkyWest Airlines
6,AS,Alaska Airlines
7,NK,Spirit Airlines
8,WN,Southwest Airlines
9,DL,Delta Air Lines


In [8]:
df = pd.read_csv("../../flight_data_raw.csv")
print('Dataframe dimensions:', df.shape)
#____________________________________________________________
# # gives some infos on columns types and number of null values
tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                          .T.rename(index={0:'null values (%)'}))
tab_info


# GLOSSARY
#https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time

Dataframe dimensions: (7213446, 26)


,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,V26
column type,int64,int64,int64,int64,object,object,object,int64,float64,float64,float64,int64,float64,float64,float64,int64,object,int64,float64,int64,float64,float64,float64,float64,float64,float64
null values (nb),0,0,0,0,0,0,0,0,112317,117234,117234,0,119245,137040,137040,0,7096862,0,134442,0,5860736,5860736,5860736,5860736,5860736,7213446
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.557051,1.625215,1.625215,0.0,1.653093,1.899785,1.899785,0.0,98.383796,0.0,1.863769,0.0,81.247382,81.247382,81.247382,81.247382,81.247382,100.0


In [9]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,V26,7213446,0.000000
1,CANCELLATION_CODE,7096862,1.616204
2,SECURITY_DELAY,5860736,18.752618
3,NAS_DELAY,5860736,18.752618
4,WEATHER_DELAY,5860736,18.752618
5,CARRIER_DELAY,5860736,18.752618
6,LATE_AIRCRAFT_DELAY,5860736,18.752618
7,ARR_DEL15,137040,98.100215
8,ARR_DELAY,137040,98.100215
9,AIR_TIME,134442,98.136231


In [10]:
#numeric features
numeric_features = [feature for feature in df.columns if df[feature].dtypes!='O']

for feature in numeric_features:
    print("Unique features for", feature, "are",len(df[feature].unique()),"out of",df.shape[0])

Unique features for YEAR are 1 out of 7213446
Unique features for MONTH are 12 out of 7213446
Unique features for DAY_OF_MONTH are 31 out of 7213446
Unique features for DAY_OF_WEEK are 7 out of 7213446
Unique features for CRS_DEP_TIME are 1369 out of 7213446
Unique features for DEP_TIME are 1441 out of 7213446
Unique features for DEP_DELAY are 1489 out of 7213446
Unique features for DEP_DEL15 are 3 out of 7213446
Unique features for CRS_ARR_TIME are 1431 out of 7213446
Unique features for ARR_TIME are 1441 out of 7213446
Unique features for ARR_DELAY are 1528 out of 7213446
Unique features for ARR_DEL15 are 3 out of 7213446
Unique features for CANCELLED are 2 out of 7213446
Unique features for DIVERTED are 2 out of 7213446
Unique features for AIR_TIME are 677 out of 7213446
Unique features for DISTANCE are 1555 out of 7213446
Unique features for CARRIER_DELAY are 1331 out of 7213446
Unique features for WEATHER_DELAY are 1038 out of 7213446
Unique features for NAS_DELAY are 953 out of 7

In [11]:
# categorical features
categorical_features = [feature for feature in df.columns if df[feature].dtypes=='O']

for feature in categorical_features:
    print("Unique categories for",feature, "are",len(df[feature].unique()),"out of",df.shape[0])

Unique categories for OP_CARRIER are 18 out of 7213446
Unique categories for ORIGIN are 358 out of 7213446
Unique categories for DEST are 358 out of 7213446
Unique categories for CANCELLATION_CODE are 5 out of 7213446


In [12]:
# Dropping redundant columns

variables_to_remove = ['V26', 'CANCELLATION_CODE']

df.drop(variables_to_remove, axis = 1, inplace = True)

In [13]:
# Removing rows where there are NAs for ARR_DELAY

print("NA values BEFORE removal:", pd.isnull(df['ARR_DELAY']).sum())
df.dropna(subset=['ARR_DELAY'],inplace=True)
print("NA values AFTER removal:", pd.isnull(df['ARR_DELAY']).sum())

NA values BEFORE removal: 137040
NA values AFTER removal: 0


## Data for Avg. Flight Delay by Airline, by Month Vis

In [21]:
df_subset = df[['YEAR', 'MONTH', 'OP_CARRIER', 'ARR_DELAY',"ORIGIN"]]

In [22]:
joined = df_subset.merge(airline_names, how = 'left', left_on = 'OP_CARRIER', right_on = 'IATA_CODE')

In [23]:
joined

,YEAR,MONTH,OP_CARRIER,ARR_DELAY,ORIGIN,IATA_CODE,AIRLINE
0,2018,4,UA,120.0,DTW,UA,United Airlines
1,2018,4,UA,-20.0,EWR,UA,United Airlines
2,2018,4,UA,10.0,SFO,UA,United Airlines
3,2018,4,UA,-13.0,TPA,UA,United Airlines
4,2018,4,UA,-4.0,SFO,UA,United Airlines
...,...,...,...,...,...,...,...
7076401,2018,9,UA,-14.0,MCI,UA,United Airlines
7076402,2018,9,UA,-13.0,DEN,UA,United Airlines
7076403,2018,9,UA,-10.0,ABQ,UA,United Airlines
7076404,2018,9,UA,7.0,IAH,UA,United Airlines


In [24]:
# Check if join is correct: OP_CARRIER == IATA_CODE
comparison_column = np.where(joined["OP_CARRIER"] == joined["IATA_CODE"], True, False)
joined["EQUAL"] = comparison_column
print("If no False values, then no missing airlines", joined["EQUAL"].unique())

missing_airlines = joined[(joined["EQUAL"] == False)]
print("If False values found, need to identify missing carrier in L_CARRIER_HISTORY.csv and update in AIRLINES.csv")
missing_airlines['OP_CARRIER'].unique()

If no False values, then no missing airlines [ True]
If False values found, need to identify missing carrier in L_CARRIER_HISTORY.csv and update in AIRLINES.csv


array([], dtype=object)

In [26]:
byCarrier = joined[['ORIGIN', 'AIRLINE', 'ARR_DELAY']]
byCarrier = byCarrier.groupby(['ORIGIN', 'AIRLINE']).mean()

In [27]:
byCarrier

ARR_DELAY
ORIGIN AIRLINE                               
ABE    Allegiant Air                11.841871
       Atlantic Southeast Airlines   7.909292
       Comair                        2.085174
       Delta Air Lines               3.875346
       Endeavor Air                  9.689459
...                                       ...
XNA    Republic Airways             14.390511
       SkyWest Airlines              8.534810
YAK    Alaska Airlines              -7.207334
YNG    Allegiant Air                75.000000
YUM    SkyWest Airlines             -5.234738

[1966 rows x 1 columns]

In [28]:
# Check for missing values in byCarrier
missing_df = byCarrier.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,ARR_DELAY,0,100.0


In [29]:
# Pivot Table View
delays_by_carrier = pd.pivot_table(byCarrier, index = 'ORIGIN', columns = 'AIRLINE', values = 'ARR_DELAY')
delays_by_carrier

AIRLINE,Alaska Airlines,Allegiant Air,American Airlines,American Eagle Airlines,Atlantic Southeast Airlines,Comair,Delta Air Lines,Endeavor Air,Frontier Airlines,Hawaiian Airlines,JetBlue Airways,Mesa Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
ORIGIN,,,,,,,,,,,,,,,,,,
ABE,NaN,11.841871,NaN,NaN,7.909292,2.085174,3.875346,9.689459,NaN,NaN,NaN,NaN,NaN,11.620133,NaN,NaN,NaN,NaN
ABI,NaN,NaN,NaN,6.535354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABQ,5.121951,15.524664,4.68686,-1.921875,3.559211,NaN,-6.656940,NaN,14.943567,NaN,4.257329,2.423353,-0.276699,4.675755,4.300464,NaN,3.093448,NaN
ABR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.835598,NaN,NaN,NaN,NaN
ABY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.819444,NaN,NaN,NaN,NaN,NaN,15.363265,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.179916,NaN,NaN,NaN,NaN
XNA,NaN,19.324232,18.04000,4.179875,7.603862,4.904691,4.378431,7.902066,NaN,NaN,NaN,8.238739,14.390511,8.534810,NaN,NaN,NaN,NaN
YAK,-7.207334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
byCarrier.to_csv('DELAY_FROM_ORIGIN_BY_CARRIER.csv', encoding='utf-8')